<a href="https://colab.research.google.com/github/nerdk312/60_Days_RL_Challenge/blob/master/Model_based_Pacman_ATARI_labels_100420_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
'''
function ClickConnect(){
console.log("Working"); 
document.querySelector("colab-toolbar-button#connect").click() 
}
setInterval(ClickConnect,60000)
'''

# Installation and Imports

In [2]:
!pip install git+git://github.com/mila-iqia/atari-representation-learning.git
!pip install git+git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail
!pip install git+git://github.com/openai/baselines
!pip install wandb

  Cloning git://github.com/mila-iqia/atari-representation-learning.git to /tmp/pip-req-build-w6bmq51o
  Running command git clone -q git://github.com/mila-iqia/atari-representation-learning.git /tmp/pip-req-build-w6bmq51o
  Created wheel for atariari: filename=atariari-0.0.1-cp36-none-any.whl size=46584 sha256=95f8e9b0993dba7a76bf1497d5c177b5f36d76163306680c1e749e90820c9947
  Stored in directory: /tmp/pip-ephem-wheel-cache-phkhfo0n/wheels/3d/69/51/5e436e5ae566c5b4dec5c53e65396d516459877a42a11d7aa4
Successfully built atariari
  Cloning git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail to /tmp/pip-req-build-nynl8rmi
  Running command git clone -q git://github.com/ankeshanand/pytorch-a2c-ppo-acktr-gail /tmp/pip-req-build-nynl8rmi
  Created wheel for a2c-ppo-acktr: filename=a2c_ppo_acktr-0.0.1-cp36-none-any.whl size=18833 sha256=3dd64c3a6824623955670ba983eef686b34c395b05be0178dcaa0227d07c9fa3
  Stored in directory: /tmp/pip-ephem-wheel-cache-bkkhf4s1/wheels/91/52/02/ec5c530fd76d56a66

In [3]:
from google.colab import drive
#drive.flush_and_unmount()
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

from __future__ import print_function
import pickle
import sys
sys.path.append('/content/gdrive/My Drive/Unsupervised_state_representation/atariari')

import wandb

import argparse

import random
import torch
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import RandomSampler, BatchSampler
from torchvision import transforms
import torchvision.transforms.functional as TF

from tqdm import tqdm
import datetime
import time

from atariari.benchmark.envs import *
from torch.autograd import Variable

import gym
from atariari.benchmark.wrapper import AtariARIWrapper

#from benchmark import *
#from methods import utils

# Needed to create dataloaders
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Imported required for the Model-based RL
from sklearn.preprocessing import StandardScaler

In [5]:
!wandb login ################

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Successfully logged in to Weights & Biases!


# Model setup

In [0]:
class EarlyStopping_loss(object):
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=5, verbose=False, wandb=None, name=""):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = 1e11 # Nawid - Set a very high initial best loss
        self.name = name
        self.wandb = wandb

    def __call__(self, val_loss, model):

        score = val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score >= self.best_score: # Nawid - Inverse signs to take into minimising loss instead of maximising accuracy
            self.counter += 1
            print(f'EarlyStopping for {self.name} counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
                print(f'{self.name} has stopped')
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(
                f'Validation loss decreased/improved for {self.name}  ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')

        save_dir = self.wandb.run.dir
        torch.save(model.state_dict(), save_dir + "/" + self.name + ".pt")
        self.wandb.save(save_dir + "/" + self.name + ".pt")
        self.val_loss_min = val_loss


In [0]:
class NNDynamicsModel(nn.Module):
    '''
    Model that predict the next state, given the current state and action
    '''
    def __init__(self, input_dim, obs_output_dim):
        super(NNDynamicsModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, obs_output_dim)
        )
    def forward(self, x):
        return self.mlp(x.float())

class NNRewardModel(nn.Module):
    '''
    Model that predict the reward given the current state and action
    '''
    def __init__(self, input_dim, reward_output_dim):
        super(NNRewardModel, self).__init__()

        self.mlp = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.BatchNorm1d(num_features=512),
            nn.ReLU(),
            nn.Linear(512,256),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(256, reward_output_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.mlp(x.float())

def model_MSEloss(y_pred,y_truth, device):
    '''
    Compute the MSE (Mean Squared Error)
    '''
    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    return F.mse_loss(y_pred.view(-1).float(), y_truth.view(-1))

def model_CEloss(y_pred,y_truth,device):
    '''
    Compute the CEloss
    '''
    y_truth = torch.Tensor(np.array(y_truth)).to(device)
    return F.cross_entropy(y_pred, y_truth)

def model_BCEloss(y_pred,y_truth,device):
    '''
    Compute the BCE (Binary cross entropy)
    param y_pred: y_pred is a 2 dimensional n x 1 data tensor
    param y_truth: y_truth is a 2D numpy array which later gets converted into a tensor
    '''
    n,c = y_pred.size()
    weights = np.zeros((n,c))
    pos = y_truth[y_truth==1].sum()
    neg = n - pos
    #nx = y_truth.cpu().data.numpy()
    index = np.where(y_truth == 1)[0]
    weights[:] = 1 #1/neg
    weights[index] = neg* 1/pos
    weights = torch.Tensor(weights).to(device)
    #print('y_pred',y_pred.size())
    #print('weights',weights.size())

    y_truth = torch.FloatTensor(np.array(y_truth)).to(device)
    #print('y_truth',y_truth.size())
    return F.binary_cross_entropy(y_pred.view(-1).float(), y_truth.view(-1),weights.view(-1))

# General Functions

In [0]:
class General_functions():
    def __init__(self, ENV_NAME, feature_size,normalise_values,n_actions,state_mode = 'ATARIARI',encoder=None):
        self.state_mode = state_mode
        self.env = AtariARIWrapper(gym.make(ENV_NAME))
        self.initial_info_labels = self.env.labels()

        self.feature_size = feature_size 
        self.normalise_values = normalise_values

        self.repeated_initial = True # Nawid - set repeated initial as true initially
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.encoder= encoder
        self.n_actions = n_actions

    def one_hot(self,i):
        a = np.zeros(self.n_actions, 'uint8')
        a[i] = 1
        return a


    def state_conversion(self,obs,info_labels):
        if self.state_mode == 'ATARIARI':
            state = []
            i = 0 
            for key in info_labels :
                if i < self.feature_size: # Nawid - Only the first 14 info are crucial I believe
                    state.append(info_labels[key])
                    i +=1
                else:
                    state = np.array(state)
                    return state
    
        elif self.state_mode == 'STDIM': # Encoding is the state
            assert self.encoder is not None
            with torch.no_grad():
                self.encoder.eval()
                state = self.encoder(obs.float().to(self.device) / 255)
                return state.cpu().detach().numpy()
        else: # Image observations are the state or the RAM labels
            return obs
    
    def check_initial(self, next_info_labels):
        # Checks if there has been a change from the initial info labels to show that the initial lag period is over 
        if self.initial_info_labels == next_info_labels:
            pass
        else:
            self.repeated_initial = False

    def reward_conversion(self,reward,done,info_labels,next_info_labels):
        # checks whether there has been a change in lives- change done to 1, otherwise the done should be fine regardless
        if not info_labels['num_lives'] == next_info_labels['num_lives']: # Nawid - If there is a change in the number of lives -  The change in the number of lives occurs after the mspacman is resurrected
            done = True 
            self.repeated_initial = True
            self.initial_info_labels  = next_info_labels # Sets the new initial labels.
        return reward, done  

    def normalise_datapoint(self,x,scaler = None):
        if self.normalise_values and not scaler == None:
            x = scaler.fit_transform(x)
        return x
    
    def inverse_normalise_datapoint(self,x, scaler = None):
        if self.normalise_values and not scaler == None:
            x = scaler.inverse_transform(x)
        return x


# Data collection and preprocessing

In [21]:
class Data_collection(General_functions):
    def __init__(self,ENV_NAME, feature_size,normalise_values,n_actions, state_model = 'ATARIARI', encoder= None):
        super(Data_collection,self).__init__(ENV_NAME,feature_size,normalise_values,n_actions, state_model, encoder)
        
    def gather_random_trajectories(self,num_traj):
        dataset_random = []
        #Env name could either be the RAM case or the generic case
        # set pacman_done as initially zero
        pacman_done = 0
        i = 0
        for n in range(num_traj):
            if n % 10 ==0:
                print('trajectory number :',n)
            # Initial set up
            obs = self.env.reset()
            self.env.seed(0)

            self.repeated_initial = True # Nawid- Used to represent the initial state
            info_labels = self.env.labels() # Nawid -  Used to get the current state
            
            while True:
                # Choosing action and env step
                
                sampled_action = np.random.randint(0,n_actions)
                sampled_action_one_hot = self.one_hot(sampled_action)
                next_obs, reward, done, next_info = self.env.step(sampled_action)
                next_info_labels = next_info['labels']
                # self.repeated initial is set to true at first and it is is turned to true in the reward_collection class when done is true
                
                self.check_initial(next_info_labels) # checks if the initial value and the current value is the same, if it is not then it changes the repeated_initial bool
                if self.repeated_initial: # If the initial state is repeating
                    pass
                        
                else:
                    # New state achieved, so save data
                    state = self.state_conversion(obs, info_labels)
                    next_state = self.state_conversion(next_obs, next_info_labels)
                    # looks if ep is done and sets the new initial info state and sets the repeated_initial bool back to true
                    reward,pacman_done = self.reward_conversion(reward, done, info_labels, next_info_labels) 
                    dataset_random.append([state, next_state,reward,pacman_done,sampled_action_one_hot])
                    
                obs = next_obs
                info_labels = next_info_labels

                # ends the loop when its done or pacman_done in the single life setting
                if done or (single_life and pacman_done):
                    break 
        
        return dataset_random

    def collate_data(self,random_dataset, rl_dataset):
        rand_data = np.array(random_dataset)
        num_rand_examples = len(rand_data)
        D_train = rand_data[:int(-num_rand_examples*1/5)] 
        D_valid = rand_data[int(-num_rand_examples*1/5):]
        print("number random examples:",num_rand_examples, 'len(D_train_rand)', len(D_train),'len(D_valid_rand)', len(D_valid))
        if len(rl_dataset) > 0:
            # Adds the rl dataset to the random dataset if there is any present
            rl_data = np.array(rl_dataset)
            num_rl_examples = len(rl_data)
            D_rl_train = rl_data[:int(-num_rl_examples*1/5)] 
            D_rl_valid = rl_data[int(-num_rl_examples*1/5):]
                        
            D_train = np.concatenate([D_train, D_rl_train], axis = 0)
            D_valid = np.concatenate([D_valid, D_rl_valid], axis = 0)
            print("number rl examples:",num_rl_examples, 'len(D_rl_train)', len(D_rl_train),'len(D_valid_rand)', len(D_rl_valid))
            
        #print("len(D_train):", len(D_train), 'len(D_valid)', len(D_valid))

        # Shuffle the dataset
        
        sff = np.arange(len(D_train))
        np.random.shuffle(sff)
        D_train = D_train[sff]
        #print('D_train shape',D_train.shape)


        # Create the input and output for the train
        X_train_obs = np.array([obs for obs,_,_,_,_ in D_train]) # Takes obs and action
        X_train_obs = X_train_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        X_train_act = np.array([act for _,_,_,_,act in D_train])
        

        # Env output
        y_env_train = np.array([no for _,no,_,_,_ in D_train])
        y_env_train = y_env_train.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)
        y_env_train = y_env_train - X_train_obs 
        #y_env_train = y_env_train - np.array([obs for obs,_,_,_,_ in D_train]) # y(state) = s(t+1) - s(t)

        # Reward's output
        y_rew_train = np.array([[D] for _,_,_,D,_ in D_train])
    
        # Next state output
        X_val_obs = np.array([obs for obs,_,_,_,_ in D_valid]) # Takes obs and action
        X_val_obs = X_val_obs.astype(np.int16) # Need to change it to a int16 so it is signed ( so negative values can be calculated)        
        X_val_act = np.array([act for _,_,_,_,act in D_valid])

        y_env_val = np.array([no for _,no,_,_,_ in D_valid])
        y_env_val = y_env_val.astype(np.int16)
        y_env_val = y_env_val - X_val_obs 
        #y_env_val = y_env_val - np.array([obs for obs,_,_,_,_ in D_valid]) # y(state) = s(t+1) - s(t)

        # Reward output
        y_rew_val = np.array([[D] for _,_,_,D,_ in D_valid])
    
        env_train_data, env_val_data = (X_train_obs, X_train_act, y_env_train), (X_val_obs, X_val_act, y_env_val)
        rew_train_data, rew_val_data = (X_train_obs, X_train_act, y_rew_train), (X_val_obs, X_val_act, y_rew_val)

        return env_train_data, env_val_data, rew_train_data, rew_val_data 

    def normalise(self,train_data, val_data,scaler = None): # Nawid - Used to normalise each dimension individually
        if self.normalise_values:
            if scaler is None:
                scaler = StandardScaler()
                train_data = scaler.fit_transform(train_data)
            else: 
                train_data = scaler.transform(train_data)        
            val_data = scaler.transform(val_data)
    
        # Normalises the values if normalise is set to true, otherwise it returns the unnormalised values
        return train_data, val_data, scaler

    def normalise_dataset(self,env_train_data, env_val_data, rew_train_data, rew_val_data,X_env_obs_scaler = None, y_env_scaler=None, X_rew_obs_scaler= None):
        # Unpack data
        (X_env_train_obs, X_env_train_act, y_env_train), (X_env_val_obs, X_env_val_act, y_env_val) = env_train_data, env_val_data
        (X_rew_train_obs, X_rew_train_act, y_rew_train), (X_rew_val_obs, X_rew_val_act, y_rew_val) = rew_train_data, rew_val_data
    
        # Normalise training and validation data
        X_env_train_obs,X_env_val_obs, X_env_obs_scaler =  self.normalise(X_env_train_obs, X_env_val_obs, X_env_obs_scaler)
        y_env_train, y_env_val, y_env_scaler = self.normalise(y_env_train, y_env_val, y_env_scaler)
        X_rew_train_obs, X_rew_val_obs, X_rew_obs_scaler = self.normalise(X_rew_train_obs, X_rew_val_obs, X_rew_obs_scaler)
    
        # Concatentates the normalised states with the one hot vector for the actions
        X_env_train = np.concatenate((X_env_train_obs,X_env_train_act),axis=1)
        X_env_val = np.concatenate((X_env_val_obs,X_env_val_act),axis=1)
        X_rew_train = np.concatenate((X_rew_train_obs,X_rew_train_act),axis=1)
        X_rew_val = np.concatenate((X_rew_val_obs,X_rew_val_act),axis=1)

        # Pack data tuples
        env_train_data, env_val_data = (X_env_train, y_env_train),(X_env_val, y_env_val) 
        rew_train_data, rew_val_data = (X_rew_train, y_rew_train),(X_rew_val, y_rew_val)

        return env_train_data, env_val_data, rew_train_data, rew_val_data, X_env_obs_scaler, y_env_scaler, X_rew_obs_scaler
    
'''
data_object = Data_collection(ENV_NAME,feature_size,normalise_on,5)
data = data_object.gather_random_trajectories(1)
rl_dataset = []
env_train, env_val, rew_train, rew_val = data_object.collate_data(data,rl_dataset)
norm_env_train, norm_env_val, norm_rew_train, norm_rew_val, X_obs_scaler, y_env_scaler, X_rew_scaler = data_object.normalise_dataset(env_train, env_val, rew_train, rew_val)
'''

trajectory number : 0
number random examples: 111 len(D_train_rand) 89 len(D_valid_rand) 22


# Controller

In [0]:
class multi_model_based_control(General_functions):
    def __init__(self,ENV_NAME, feature_size,normalise_values,n_actions,norm_scalers,env_model,rew_model, num_sequences,horizon_length, state_model = 'ATARIARI', encoder= None):
        super(multi_model_based_control,self).__init__(ENV_NAME,feature_size,normalise_values,n_actions, state_model, encoder)
        
        self.env_model = env_model
        self.rew_model = rew_model
        self.horizon_length = horizon_length
        self.num_sequences = num_sequences        
        self.n_actions = n_actions

        if self.normalise_values == False:
            self.env_input_scaler = None
            self.env_output_scaler = None
            self.rew_input_scaler = None
        else:
            self.env_input_scaler, self.env_output_scaler, self.rew_input_scaler = norm_scalers
        print('normalise_values :',self.normalise_values)

    def random_sampling_shooting(self,real_obs):
        '''
        Use a random-sampling shooting method, generating random action sequences. The first action with the highest reward of the entire sequence is returned
        '''
        best_reward = -1e9
        best_next_action = []
        m_obs = np.array([real_obs for _ in range(self.num_sequences)])

        # array that contains the rewards for all the sequence
        unroll_rewards = np.zeros((self.num_sequences, 1)) 
        first_sampled_actions = []

        self.env_model.eval()
        self.rew_model.eval()

        # Create a batch of size 'num_sequences' (number of trajectories) to roll the models 'horizon length' times
        ## i.e roll a given number of trajectories in a single batch (to increase speed)
        for t in range(self.horizon_length):
            # sample actions for each sequence
            sampled_actions = np.array([np.random.randint(0,self.n_actions) for _ in range(self.num_sequences)])
            sampled_actions_one_hot = np.array([self.one_hot(action) for action in sampled_actions])
            if isinstance(self.env_model,NNDynamicsModel): # Nawid-  If the env model is a neural net
                #print('using dynamics model')
                print('env input scaler:', self.env_input_scaler)
                m_obs_env_scaled = self.normalise_datapoint(m_obs,self.env_input_scaler) # The env_input scaler should be none when normalisation is off
                print('m obs env_scaled',m_obs_env_scaled)
                env_model_input = np.concatenate([m_obs_env_scaled, sampled_actions_one_hot], axis = 1)
                # compute the next state for each sequence
                pred_obs = self.env_model(torch.tensor(env_model_input).to(self.device))

                # inverse scaler transformation
                print('env output scaler', self.env_output_scaler)
                pred_obs = self.inverse_normalise_datapoint(pred_obs.cpu().detach().numpy(),self.env_output_scaler)
                print('pred obs', pred_obs)
                # add previous observation
                next_obs = pred_obs + m_obs
            else:
                #print('using oracle state')            
                next_obs = self.env_model.predict_states(m_obs,sampled_actions) # Nawid - Able to obtain the next state directly rather than predicting a change in states

            if isinstance(self.rew_model, NNRewardModel):
                print('rew input_scaler', self.rew_input_scaler)
                m_obs_rew_scaled = self.normalise_datapoint(m_obs, self.rew_input_scaler)
                print('m obs rew scaled', m_obs_rew_scaled)
                rew_model_input = np.concatenate([m_obs_rew_scaled, sampled_actions_one_hot], axis = 1)
                pred_rew = self.rew_model(torch.tensor(rew_model_input).to(self.device)) # Nawid -  I believe I do not need to rescale for a True of false situation
                unroll_rewards += (1 - pred_rew.cpu().detach().numpy())
            else:
                #print('using oracle reward')
                pred_rew = self.rew_model.predict_reward(m_obs, sampled_actions,next_obs)
                unroll_rewards += pred_rew
        
            m_obs = next_obs # Nawid - Update the state after calculating the new state and calculating the rewards

            if t ==0:
                first_sampled_actions = sampled_actions
        
        self.env_model.train()
        self.rew_model.train()

        # Best the position of the sequence with the higher reward
        arg_best_reward = np.argmax(unroll_rewards)
        best_sum_reward = unroll_rewards[arg_best_reward].squeeze()
        # take the first action of this sequence
        best_action = first_sampled_actions[arg_best_reward]
        #best_action =  np.squeeze(best_action)
        return best_action, best_sum_reward

# Train model

In [0]:
def train_model(train_data,val_data,model,batch_size, max_model_iter, optimizer, device,early_stopper,desired_model='Env_model'):
    ''' 
    General function to train either of the two models
    '''
    # Unpack data
    (X_train, y_train), (X_val, y_val) =  train_data, val_data
    losses_env = []

    # Choose loss function based on what type of model is training
    if desired_model =='Env_model': # Nawid -  Decides which loss function to use
        loss_function = model_MSEloss
    else:
        loss_function = model_BCEloss
        #print('BCE being used')

    # go through max_model iter supervised iterations
    for it in tqdm(range(max_model_iter)):
        # create mini batches of size batch_size
        for mb in range(0,len(X_train), batch_size): # Nawid- Batch size is the step size
            if len(X_train) > mb + BATCH_SIZE:
                X_mb = X_train[mb:mb+BATCH_SIZE]
                y_mb = y_train[mb:mb+BATCH_SIZE]
                #X_mb += np.random.normal(loc = 0, scale = 0.001, size= X_mb.shape)

                optimizer.zero_grad()
                # forward pass of model to compute the output
                pred_mb = model(torch.tensor(X_mb).to(device))
                
                # compute the loss
                loss = loss_function(pred_mb,y_mb,device) #Nawid-  Uses Mse loss if dynamics model or uses BCE loss if reward/done model
                wandb.log({'{} Training loss'.format(desired_model):loss.cpu().detach().numpy()})
                # backward pass
                loss.backward()
                # optimization step
                optimizer.step()
        
        # Nawid - Calculate the validation loss after each epoch
        model.eval()
        with torch.no_grad():
            pred_val = model(torch.tensor(X_val).to(device))
            val_loss = loss_function(pred_val, y_val,device)
            wandb.log({'{} Validation loss'.format(desired_model):val_loss})
        

        # Checks whether to early stop after each epoch
        early_stopper(val_loss,model)
        if early_stopper.early_stop:
            print("Early stopping")
            break

# Hyperparameters

In [29]:
wandb.init(entity="nerdk312", project="ATARI_LABELS_MPC")

W&B Run: https://app.wandb.ai/nerdk312/ATARI_LABELS_MPC/runs/2kt0mhkt

In [0]:
#ENV_NAME = 'MsPacmanNoFrameskip-v4'
ENV_NAME = 'MsPacmanDeterministic-v4'
feature_size = 10
#feature_size = all_defaults['feature_size'] # Nawid- Dimensionality of the representation
# workers = 8 # Nawid - Choosing the number of workers for the network

# Main loop hyperp
env_model_pretrain =  False
rew_model_pretrain = False
if env_model_pretrain:
    pretrained_env_model = '/content/gdrive/My Drive/MsPacman-data/Env_model_4_9.47.41.pt'

if rew_model_pretrain:
    pretrained_rew_model = '/content/gdrive/My Drive/MsPacman-data/Rew_model_1000_randtraj_lr1e-4.pt'
    AGGR_ITER = 10
else:
    AGGR_ITER = 100

STEPS_PER_AGGR = 100

# Random MB hyperp
NUM_RAND_TRAJECTORIES = 1

# 'cuda' or 'cpu'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Supervised Model Hyperp
ENV_LEARNING_RATE = 1e-3
REW_LEARNING_RATE = 1e-4
BATCH_SIZE = 1024
TRAIN_ITER_MODEL =  100

# Controller Hyperp
HORIZON_LENGTH = 5
NUM_ACTIONS_SEQUENCES = 200

load_data = False
if load_data:
    loaded_trajectories = '/content/gdrive/My Drive/MsPacman-data/pacman_rand1000.npy'

collect_data = False
observation_channels = 1
action_dim = 1
n_actions = 5 #9 - Nawid - Change to 5 actions as the 4 other actions are simply copies of the other actions, therefore 5 actions should lower the amount of data needed.
reward_dim = 1

# Decides if an episode is done or not
single_life = True

# Time and date information
now = datetime.datetime.now()
date_time = "{}_{}_{}-{}_{}".format(now.day,now.month,now.year, now.hour, now.minute)

# Making the network deterministic - https://pytorch.org/docs/stable/notes/randomness.html
random_seed = 0
normalise_on = False
if random_seed:    
    torch.manual_seed(1)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    #np.random.seed(0)

# Saving Config file

In [0]:
config = wandb.config
config.batch_size = BATCH_SIZE          
config.horizon_length = HORIZON_LENGTH
config.num_action_seq = NUM_ACTIONS_SEQUENCES
config.train_model_iter = TRAIN_ITER_MODEL 
config.num_rand_trajectories = NUM_RAND_TRAJECTORIES
config.aggr_iter = AGGR_ITER
config.steps_per_aggr = STEPS_PER_AGGR
config.env_lr = ENV_LEARNING_RATE
config.rew_lr = REW_LEARNING_RATE
config.no_actions = n_actions
config.load_data = load_data
config.collect_data = collect_data
config.env_model_pretrain = env_model_pretrain
config.rew_model_pretrain = rew_model_pretrain
config.single_life = single_life
config.random_seed = random_seed

if load_data:
    config.loaded_trajectories = loaded_trajectories

if env_model_pretrain:
    config.pretrained_env_model = pretrained_env_model
if rew_model_pretrain:
    config.pretrained_rew_model = pretrained_rew_model

# Main - Model training

Data collection and processing

In [62]:
# gather the dataset of random sequences
data_collector = Data_collection(ENV_NAME,feature_size,normalise_on,n_actions)

if load_data:
    rand_dataset = np.load(loaded_trajectories,allow_pickle=True)
else:
    rand_dataset = data_collector.gather_random_trajectories(NUM_RAND_TRAJECTORIES)
    filename = '/content/gdrive/My Drive/MsPacman-data/rand_traj_{}-'.format(NUM_RAND_TRAJECTORIES) + date_time
    np.save(filename,rand_dataset)

rl_dataset = []
env_train_data, env_val_data, rew_train_data, rew_val_data = data_collector.collate_data(rand_dataset,rl_dataset)

norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = data_collector.normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data)
norm = (X_env_state_scaler, y_env_scaler, X_rew_state_scaler)


trajectory number : 0
number random examples: 91 len(D_train_rand) 73 len(D_valid_rand) 18


Dynamics model training

In [0]:
env_model = NNDynamicsModel(n_actions + feature_size, feature_size).to(device) # Nawid - Need to put both actions as it is a one-hot vector

if env_model_pretrain:
    env_model.load_state_dict(torch.load(pretrained_env_model))
    env_model.eval()
    
else:
    env_optimizer = torch.optim.Adam(env_model.parameters(),ENV_LEARNING_RATE) # Nawid - Optimizer for the env model
    wandb.watch(env_model, log="all")
    env_model_name = 'Env_model'+ '_' + date_time
    early_stopping_env = EarlyStopping_loss(patience=5, verbose=True, wandb=wandb, name=env_model_name)
    
    for n_iter in range(AGGR_ITER):
        if early_stopping_env.early_stop:
            print('Early stopping')
            break
        train_model(norm_env_train_data, norm_env_val_data, env_model, BATCH_SIZE, TRAIN_ITER_MODEL,env_optimizer,device,early_stopping_env)

Training the reward function and MPC

In [0]:

rew_model = NNRewardModel(n_actions + feature_size, reward_dim).to(device) # Nawid - Need to put both actions as it is a one-hot vector

if rew_model_pretrain:
    rew_model.load_state_dict(torch.load(pretrained_rew_model))

rew_optimizer = torch.optim.Adam(rew_model.parameters(),REW_LEARNING_RATE) # Nawid - Optimizer for the env model
wandb.watch(rew_model, log="all")
Rew_model_name = 'Rew_model'+ '_' + date_time
early_stopping_rew = EarlyStopping_loss(patience=5, verbose=True, wandb=wandb, name=Rew_model_name)


Controller = multi_model_based_control(ENV_NAME,feature_size, normalise_on,n_actions,norm,env_model, rew_model, NUM_ACTIONS_SEQUENCES, HORIZON_LENGTH)
global_step = 0
for n_iter in range(AGGR_ITER):
    if early_stopping_rew.early_stop:
        print('Early stopping')
        break

    if not rew_model_pretrain:
        train_model(norm_rew_train_data, norm_rew_val_data, rew_model, BATCH_SIZE, TRAIN_ITER_MODEL,rew_optimizer,device,early_stopping_rew,desired_model='Rew_model')

    
    if collect_data:
        env_train_data, env_val_data, rew_train_data, rew_val_data = data_collector.collate_data(rand_dataset,rl_dataset)
        norm_env_train_data, norm_env_val_data, norm_rew_train_data, norm_rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler = normalise_dataset(env_train_data, env_val_data, rew_train_data, rew_val_data, X_env_state_scaler, y_env_scaler, X_rew_state_scaler)

    obs = Controller.env.reset()
    Controller.env.seed(0) # Set the random seed of the environment
    #obs = env.reset()
    #initial_info_labels = env.labels()

    num_examples_added = 0
    game_reward = 0
    # records how long the agent survives for
    timesteps = 0
    controller_pred_rews = []
    rews = []
    # records how many simulations have occurred
     
    #i = 0 
    while num_examples_added < STEPS_PER_AGGR:
        while True:
            tt = time.time()
            
            if Controller.repeated_initial: # Placed the repeated initial before the check_initial info labels since if the actions were chosen by MPC for the 264 intial actions, it would increase the computational time by quite a lot
                #i += 1
                #print(i)
                obs,_,_,info = Controller.env.step(0) # Any action taken
                info_labels = info['labels']
                # Checks if the initial set is being repeated and if it isnt, it sets the value off
                Controller.check_initial(info_labels) 
            else:
                # new state achieved
                state = Controller.state_conversion(obs,info_labels)
                action, pred_rew = Controller.random_sampling_shooting(state)
                action_one_hot = Controller.one_hot(action)
                controller_pred_rews.append(pred_rew)

                # one step in the environment with the action returned by
                next_obs, reward,done, next_info = Controller.env.step(action)
                next_info_labels = next_info['labels']
                
                next_state = Controller.state_conversion(next_obs, next_info_labels)
                # Obtains the reward, done and sets whether repeated initial should be true or not
                reward, pacman_done = Controller.reward_conversion(reward,done,info_labels, next_info_labels) 

                # add to the RL dataset                
                rl_dataset.append([state, next_state, reward, pacman_done, action_one_hot])

                num_examples_added += 1
                timesteps +=1 
                obs = next_obs
                info_labels = next_info_labels

                game_reward += reward 
                if done or (single_life and pacman_done):
                    global_step += 1
                    obs = Controller.env.reset()
                    Controller.env.seed(0) # Need to set the random seed after the environment is done
                    wandb.log({'game reward':game_reward, 'pred_rew': np.mean(controller_pred_rews),'survival time':timesteps,'global step':global_step })
                    print('  >> R: {:.2f}, Mean sum:{:.2f},Survival time: {},Num examples:{}'.format(game_reward, np.mean(controller_pred_rews),timesteps,num_examples_added))
                    rews.append(game_reward)
                    game_reward = 0
                    timesteps = 0
                    controller_pred_rews = []
                    break

    print('  >> Mean: {:.2f}', np.mean(rews))    